In [ ]:
#| default_exp main

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import os, torch, scipy.sparse as sp, joblib, argparse
from typing import Optional, Dict, Callable, Union

from xcai.block import SXCBlock, XCBlock
from xcai.core import get_best_model, load_config

## `Arguement`

In [ ]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--build_block', action='store_true')
    parser.add_argument('--use_pretrained', action='store_true')
    
    parser.add_argument('--do_train_inference', action='store_true')
    parser.add_argument('--do_test_inference', action='store_true')
    
    parser.add_argument('--save_train_inference', action='store_true')
    parser.add_argument('--save_test_inference', action='store_true')
    
    parser.add_argument('--save_representation', action='store_true')
    
    parser.add_argument('--use_sxc_sampler', action='store_true')
    parser.add_argument('--only_test', action='store_true')

    parser.add_argument('--pickle_dir', type=str, required=True)
    
    return parser.parse_args()
    

## Build `block`

In [ ]:
#| export
def build_block(pkl_file:str, config:Union[str,Dict], use_sxc:Optional[bool]=True, config_key:Optional[str]=None, 
                do_build:Optional[bool]=False, only_test:Optional[bool]=False, **kwargs):

    if not os.path.exists(pkl_file): do_build = True

    if do_build:
        if isinstance(config, str) and os.path.exists(config): 
            config = load_config(config, config_key)
            if only_test and 'train' in config['path']: del config['path']['train'] 
    
        if use_sxc: 
            block = SXCBlock.from_cfg(config, padding=True, return_tensors='pt', **kwargs)
        else: 
            block = XCBlock.from_cfg(config, transform_type='xcs', **kwargs)
            
        joblib.dump(block, pkl_file)
    else:
        block = joblib.load(pkl_file)

    return block
        

## Load `model`

In [ ]:
#| export
def load_model(output_dir:str, model_fn:Callable, model_args:Dict, init_fn:Callable, init_args:Optional[Dict]=dict(), 
               do_inference:Optional[bool]=False, use_pretrained:Optional[bool]=False):
    if do_inference and not use_pretrained:
        os.environ['WANDB_MODE'] = 'disabled'
        model_args['mname'] = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

    model = model_fn(**model_args)
    
    if not do_inference or use_pretrained: 
        init_fn(model, **init_args)

    return model
    

## `main` function

In [ ]:
#| export
def get_output(pred_idx:torch.Tensor, pred_ptr:torch.Tensor, pred_score:torch.Tensor, n_lbl:int, **kwargs):
    n_data = pred_ptr.shape[0]
    pred_ptr = torch.cat([torch.zeros((1,), dtype=torch.long), pred_ptr.cumsum(dim=0)])
    pred = sp.csr_matrix((pred_score,pred_idx,pred_ptr), shape=(n_data, n_lbl))
    return pred
    

In [ ]:
#| export
def main(learn, args, n_lbl:int):
    do_infer = args.do_train_inference or args.do_test_inference or args.save_train_inference \
            or args.save_test_inference or args.save_representation
    
    if do_infer:
        pred_dir = f'{learn.args.output_dir}/predictions'
        os.makedirs(pred_dir, exist_ok=True)

        if args.save_representation:
            trn_repr, lbl_repr = learn.get_data_and_lbl_representation(learn.train_dataset)
            tst_repr = learn._get_data_representation(learn.eval_dataset)

            torch.save(trn_repr, f'{pred_dir}/train_repr.pth')
            torch.save(tst_repr, f'{pred_dir}/test_repr.pth')
            torch.save(lbl_repr, f'{pred_dir}/label_repr.pth')

        if args.do_test_inference:
            o = learn.predict(learn.eval_dataset)
            print(o.metrics)

            if args.save_test_inference:
                with open(f'{pred_dir}/test_predictions.pkl', 'wb') as file:
                    pickle.dump(o, file)

                pred = get_output(o.pred_idx, o.pred_ptr, o.pred_score, n_lbl=n_lbl)
                sp.save_npz(f'{pred_dir}/test_predictions.npz', pred)

        if args.do_train_inference:
            o = learn.predict(learn.train_dataset)
            print(o.metrics)

            if parse_args.save_train_inference:
                with open(f'{pred_dir}/train_predictions.pkl', 'wb') as file:
                    pickle.dump(o, file)

                pred = get_output(o.pred_idx, o.pred_ptr, o.pred_score, n_lbl=n_lbl)
                sp.save_npz(f'{pred_dir}/train_predictions.npz', pred)
    else:
        learn.train()
        